
## EE 342:  Lab 2 - Elementary Music Synthesis

Instructor: Prof. Amy Orsborn

Teaching Assistant: Christopher Yin

**Overview**
The purpose of this lab is to familiarize you with constructing and processing discrete-time sound signals. You will learn how to synthesize music notes and play them in in Python $+$ SciPy environment. You will concatenate a series of music notes into a small music piece, add volume variation to the music piece, overlap the adjacent notes to further make the music smoother and more realistic.

Through this process, you will learn to perform the following sound synthesis tasks:
- Generating musical notes
- Improving the quality of the generated sound
- Overlapping notes

**Team Members**: Caitlin and Aya



In [ ]:
!pip install sounddevice
!sudo apt-get install libportaudio2
import numpy as np
import sounddevice as sd
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  libportaudio2
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 65.4 kB of archives.
After this operation, 223 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libportaudio2 amd64 19.6.0-1build1 [65.4 kB]
Fetched 65.4 kB in 0s (167 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open

In [ ]:
# SimpleAudio constants.
DEFAULT_SAMPLE_RATE = 8000
DEFAULT_AMPLITUDE = 1

# Musical note frequencies.
A = 220.0
B = A * 2 ** (2 / 12)
C = A * 2 ** (3 / 12)
E = A * 2 ** (7 / 12)

## Task 1: Generating Musical Notes

In [ ]:
def play(data, sample_rate=DEFAULT_SAMPLE_RATE):
    """Plays audio data.

    Args:
        data: The audio data to be played (as a NumPy array).
        sample_rate: The sample rate.
    """
    sd.play((data * 32767).astype('int16'), sample_rate)

In [ ]:
# defining note function 
def note(frequency, duration, amplitude=DEFAULT_AMPLITUDE, sample_rate=DEFAULT_SAMPLE_RATE):
    """Generates samples of a musical note.

    Samples are generated according to following equation:

        y(t) = A sin(wt)

    Args:
        frequency: The frequency of the note in Hz.
        duration: The length of the note in seconds.
        amplitude: The amplitude of the note.
        sample_rate: The sample rate in Hz.

    Returns:
        An array of samples representing the note.
    """
    t = np.linspace(0, duration, int(duration * sample_rate))
    note = amplitude * np.sin(2 * np.pi * frequency * t)
    return note

In [ ]:
# Use your function to generate the musical score for the Scarborough Fair.

#Rests corresponding to counts, for note types as well as a standard rest. Might change.
rest = np.zeros(int(500))

#Array of notes to form song.
score = [
    (A, 1.0),
    (A, 0.5),
    (E, 0.5),
    (E, 0.5),
    (E, 0.5),
    (B, 0.5),
    (C, 0.5),
    (B, 0.5),
    (A, 2.0)
]

#Concatenates the new note to the song.
song = np.array([])
for frequency, duration in score:
    this_note = note(frequency, duration)
    song = np.concatenate([song, this_note, rest])

# Play the audio sample
play(song)

PortAudioError: ignored

## Task 2: Volume Variations

In [ ]:
### This code is provided to you. ###

def display_envelope(signal,duration=-1,sample_rate=DEFAULT_SAMPLE_RATE):

    """
    Plots the envelope of a duration of the generated audio. 

    Args:
        signal: The audio signal to be displayed (as a NumPy array).
        duration: The length of hte signal to be displayed. -1 to display entire signal.
        sample_rate: The sample rate.
    """
    
    from scipy.signal import hilbert
    
    if duration < 0:
        duration = len(signal)/sample_rate
    
    n = int(duration*sample_rate)
    
    analytic_signal = hilbert(signal[0:n])
    amplitude_envelope = np.abs(analytic_signal)
        
    t = np.linspace(0,duration,len(amplitude_envelope))
    
    plt.figure()    
    plt.plot(t,amplitude_envelope)
    plt.show()

In [ ]:
# Volume Variation Function Attack, Decay, Sustain and Release (ADSR)
def adsr(note):
    """Generates an ADSR envelope and applies it to a note.

        A: Linear increase to 60% amplitude over 20% length.
        D: Linear decrease to 50% amplitude over 4% length.
        S: Constant 50% amplitude over 40% length.
        R: Linear decrease to 0% amplitude over 36% length.

    Args:
        note: The note to be modified.

    Returns:
        A note that has been scaled by the ADSR envelope.
    """
    x = len(note)
    ADSR_A = np.arange(0, 0.6, 0.6 / (x * 0.20))
    ADSR_D = 1 - np.arange(0, 0.5, 0.5 / (x * 0.04))[::-1]
    ADSR_S = np.full(int(x * 0.40), 0.5)
    ADSR_R = 1 - np.arange(0, 0.0, 0.01 / (x * 0.36))[::-1]

    t = np.concatenate([ADSR_A, ADSR_D, ADSR_S, ADSR_R])

    #Concatenates the song
    ADSR_song = np.zeros(len(t))
    for i in range(min(len(note), len(t))):
      ADSR_song[i] = t[i] * note[i]    
  
    return ADSR_song

In [ ]:
# Use your function Apply an ADSR envelope to each note.
ADSR_score = [
    adsr(note(A, 1.0)),
    adsr(note(A, 0.5)),
    adsr(note(E, 0.5)),
    adsr(note(E, 0.5)),
    adsr(note(E, 0.5)),
    adsr(note(B, 0.5)),
    adsr(note(C, 0.5)),
    adsr(note(B, 0.5)),
    adsr(note(A, 2.0))
]

# Play the audio sample
play(song)

# Display the envelope
display_envelope(song)

PortAudioError: ignored

**Discussion Question:** Instead of enveloping with the ADSR function, how would you envelope with a decaying exponential?

To envelope with a decaying exponential, I would multiply each note by a decaying exponential function pver a specified duration. The duration and rate of decay of the exponential function would determine the shape and length of the envelope.

**Optional:** Implement a decaying exponential envelope.

## Task 3: Overlapping Notes

In [ ]:
def overlap(notes, offset):
    """Joins notes together with overlap between consecutive notes.

    Args:
        notes: An array of notes to be joined.
        offset: The amount of allowable overlap between consecutive notes.

    Returns:
        A score made from the input notes.
    """
    # Enter code here
    total_len = 0
    for i in notes:
        total_len += len(i)

    score_len = total_len - ((len(notes)-1)*offset) #score length removes the indexes corresponding to the overlapping areas
    score = np.zeros(score_len) 

    j = 0
    for note in notes:
        if j != 0: #skips the first note
            j = j-offset #once the first note is skipped, adjusts the array indexing by the offset for that note
            for n in note:
              score[j] = score[j]+n #adds the note values at the offsetted indices
              j+=1 #increases index by one

    return score

In [20]:
# Use your function Apply an ADSR envelope to each note.
ADSR_score = [
    adsr(note(A, 1.0)),
    adsr(note(A, 0.5)),
    adsr(note(E, 0.5)),
    adsr(note(E, 0.5)),
    adsr(note(E, 0.5)),
    adsr(note(B, 0.5)),
    adsr(note(C, 0.5)),
    adsr(note(B, 0.5)),
    adsr(note(A, 2.0))
]

overlapped = overlap(ADSR_score, 400)

# Play the audio sample
play(overlapped)

# Display the envelope
display_envelope(overlapped)

PortAudioError: ignored